In [ ]:
import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

from matplotlib import pyplot as plt
import numpy as np
from torch import nn
import torch
import config

from utils import show_abundance, plot_endmembers
# from dataset import JasperRidgeDataset
from HySpecLab.metrics import rmse, sad
from scipy import io as sio

In [ ]:
from dataset import JasperRidge

dataset = JasperRidge(config.JasperRidge_PATH)
dataset_name = 'JasperRidge'
wv = np.array(dataset.wv, dtype=np.uint)

result_path = os.path.join(config.RESULTS_PATH, 'jasperRidge')

# Ground Truth

In [ ]:
from utils import plot_endmembers, show_abundance
fig = plot_endmembers(dataset.endmembers(), wv, ticks_range=(0, 1), n_ticks=5)
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/M_ref.pdf'), bbox_inches='tight')

fig = show_abundance(dataset.abundance())
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/A_ref.png'), dpi=300, bbox_inches='tight')

In [ ]:
from HySpecLab.metrics import sad

def sort_endmember(endmembers, gt):
    sad_result = sad(endmembers, gt)
    e_idx = torch.argmin(sad_result, dim=0) # Index for reordering the ground truth
    return e_idx, sad_result

In [ ]:
from HySpecLab.eea import VCA

n_endmembers = dataset.n_endmembers
   
vca = VCA(n_endmembers, snr_input=-1, random_state=25)
vca.fit(dataset.X.numpy())
endmembers = torch.from_numpy(vca.endmembers()).float()
e_idx, sad_result = sort_endmember(endmembers, dataset.endmembers())

vca_endmember_init = endmembers[e_idx]
vca_logit_endmember_init = torch.log((vca_endmember_init + 1e-12) / ((1-vca_endmember_init) + 1e-12))

fig = plot_endmembers(vca_endmember_init, wv, ticks_range=(0, 1))
plt.show(fig)

# fig.savefig(os.path.join(result_path, 'imgs/M_vca.pdf'), bbox_inches='tight')

In [ ]:
from utils import plot_endmembers
from pysptools import eea
n_endmembers = dataset.n_endmembers

ee = eea.NFINDR()
endmember = torch.from_numpy(ee.extract(dataset.image(), n_endmembers)).float()

e_idx, _ = sort_endmember(endmember, dataset.endmembers())
nfindr_endmember_init = endmember[e_idx]
nfindr_logit_endmember_init = torch.log((nfindr_endmember_init + 1e-12) / ((1-vca_endmember_init) + 1e-12))

fig = plot_endmembers(nfindr_endmember_init, wv, ticks_range=(0, 1))
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/M_nfindr.pdf'), bbox_inches='tight')

In [ ]:
fig = plot_endmembers(dataset.endmembers() / dataset.endmembers().max(), wv, ticks_range=(0, 1), endmember_estimation=[nfindr_endmember_init, vca_endmember_init], ee_labels=['Ground Truth', 'N-FINDR', 'VCA'])
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/M_estimation.pdf'), bbox_inches='tight')

In [ ]:
# endmember_init_method = 'nfindr'
# endmember_init = nfindr_endmember_init
# logit_endmember_init = nfindr_logit_endmember_init

endmember_init_method = 'vca'
endmember_init = vca_endmember_init
logit_endmember_init = vca_logit_endmember_init

In [ ]:
param_grid = {
    'sigma_sparsity': [.05, .1, .25, .5],
    'sparse_weight': [0, .05, .1, .25, .5, 1],
}

In [ ]:
from torch import sigmoid
def test(model, dataset):
    X = dataset.X
    model.eval()
    model = model.cpu()
    
    X_true = dataset.A @ dataset.endmembers()
    with torch.no_grad():
        X_hat = model(dataset.X)
        A_hat = torch.softmax(model.A, dim=1)
        M_hat = sigmoid(model.ebk) 
    
    rmse_x = rmse(X_true, X_hat, dim=None).numpy()      
    rmse_a = rmse(dataset.A, A_hat, dim=None).numpy()
    sad_m = np.diagonal(sad(M_hat, dataset.endmembers()).numpy()).mean()
    return rmse_x.item(), rmse_a.item(), sad_m

In [ ]:
from sklearn.model_selection import ParameterGrid
from utils import train 
from HySpecLab.unmixing import ContrastiveUnmixing
import pandas as pd
from tqdm import tqdm

n_bands = dataset.n_bands
results = pd.DataFrame(columns=['sigma_sparsity', 'sparse_weight', 'mean_rmse_x', 'mean_rmse_a', 'mean_sad_m'])
for params in ParameterGrid(param_grid):
    sigma = params['sigma_sparsity']
    sparse_weight = params['sparse_weight']
    batch_rmse_x, batch_rmse_a, batch_sad_m = [], [], []

    for i in tqdm(range(5)):
        model = ContrastiveUnmixing(n_bands, n_endmembers, endmember_init=logit_endmember_init, sigma_sparsity=sigma)
        train(model, n_endmembers, dataset, n_batchs=50, n_epochs=100, lr=1e-3, similarity_weight=1, sparse_weight=sparse_weight)
        rmse_x, rmse_a, sad_m = test(model, dataset)
        batch_rmse_x.append(rmse_x)
        batch_rmse_a.append(rmse_a)
        batch_sad_m.append(sad_m)

    mean_rmse_x = sum(batch_rmse_x) / len(batch_rmse_x)
    mean_rmse_a = sum(batch_rmse_a) / len(batch_rmse_a)
    mean_sad_m = sum(batch_sad_m) / len(batch_sad_m)
    
    results = results.append({'sigma_sparsity': sigma, 'sparse_weight': sparse_weight, 'mean_rmse_x': mean_rmse_x, 'mean_rmse_a': mean_rmse_a, 'mean_sad_m': mean_sad_m}, ignore_index=True)

In [ ]:
csv_save_path = os.path.join(result_path, 'hyperparameters.csv')
results.to_csv(csv_save_path, index=False)

# Apex

In [ ]:
from dataset import Apex
dataset = Apex(config.Apex_PATH)

result_path = os.path.join(config.RESULTS_PATH, 'apex')

In [ ]:

fig = plot_endmembers(dataset.endmembers(), np.array(dataset.wv), ticks_range=(0, .5), n_ticks=5)
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/M_ref.pdf'), bbox_inches='tight')

fig = show_abundance(dataset.abundance())
plt.show(fig)


In [ ]:
from HySpecLab.eea import VCA

n_endmembers = dataset.n_endmembers
   
vca = VCA(n_endmembers, snr_input=20, random_state=42)
vca.fit(dataset.X.numpy())
endmembers = torch.from_numpy(vca.endmembers()).float()
e_idx, sad_result = sort_endmember(endmembers, dataset.endmembers())

vca_endmember_init = endmembers[e_idx]
vca_logit_endmember_init = torch.log((vca_endmember_init + 1e-12) / ((1-vca_endmember_init) + 1e-12))

fig = plot_endmembers(vca_endmember_init, dataset.wv, ticks_range=(0, 1))
plt.show(fig)

endmember_init_method = 'vca'
endmember_init = vca_endmember_init
logit_endmember_init = vca_logit_endmember_init

In [ ]:
from sklearn.model_selection import ParameterGrid
from utils import train 
from HySpecLab.unmixing import ContrastiveUnmixing
import pandas as pd
from tqdm import tqdm

n_bands = dataset.n_bands
results = pd.DataFrame(columns=['sigma_sparsity', 'sparse_weight', 'mean_rmse_x', 'mean_rmse_a', 'mean_sad_m'])
for params in ParameterGrid(param_grid):
    sigma = params['sigma_sparsity']
    sparse_weight = params['sparse_weight']
    print('Sigma:', sigma, 'Sparse Weight:', sparse_weight)
    batch_rmse_x, batch_rmse_a, batch_sad_m = [], [], []

    for i in tqdm(range(3)):
        model = ContrastiveUnmixing(n_bands, n_endmembers, endmember_init=logit_endmember_init, sigma_sparsity=sigma)
        train(model, n_endmembers, dataset, n_batchs=50, n_epochs=100, lr=1e-3, similarity_weight=1, sparse_weight=sparse_weight)
        rmse_x, rmse_a, sad_m = test(model, dataset)
        batch_rmse_x.append(rmse_x)
        batch_rmse_a.append(rmse_a)
        batch_sad_m.append(sad_m)

    mean_rmse_x = sum(batch_rmse_x) / len(batch_rmse_x)
    mean_rmse_a = sum(batch_rmse_a) / len(batch_rmse_a)
    mean_sad_m = sum(batch_sad_m) / len(batch_sad_m)
    
    results = results.append({'sigma_sparsity': sigma, 'sparse_weight': sparse_weight, 'mean_rmse_x': mean_rmse_x, 'mean_rmse_a': mean_rmse_a, 'mean_sad_m': mean_sad_m}, ignore_index=True)
    print('Mean RMSE X:', mean_rmse_x, 'Mean RMSE A:', mean_rmse_a, 'Mean SAD M:', mean_sad_m)

In [ ]:
results

In [ ]:
csv_save_path = os.path.join(result_path, 'hyperparameters.csv')
results.to_csv(csv_save_path, index=False)

# Samson

In [ ]:
from dataset import Samson

dataset = Samson(config.Samson_PATH)
result_path = os.path.join(config.RESULTS_PATH, 'samson')

In [ ]:
fig = plot_endmembers(dataset.endmembers(), np.array(dataset.wv), ticks_range=(0, .5), n_ticks=5)
plt.show(fig)

fig = show_abundance(dataset.abundance())
plt.show(fig)

In [ ]:
from HySpecLab.eea import VCA

n_endmembers = dataset.n_endmembers
   
vca = VCA(n_endmembers, snr_input=20, random_state=42)
vca.fit(dataset.X.numpy())
endmembers = torch.from_numpy(vca.endmembers()).float()
e_idx, sad_result = sort_endmember(endmembers, dataset.endmembers())

vca_endmember_init = endmembers[e_idx]
vca_logit_endmember_init = torch.log((vca_endmember_init + 1e-12) / ((1-vca_endmember_init) + 1e-12))

fig = plot_endmembers(vca_endmember_init, dataset.wv, ticks_range=(0, 1))
plt.show(fig)

endmember_init_method = 'vca'
endmember_init = vca_endmember_init
logit_endmember_init = vca_logit_endmember_init

In [ ]:
from sklearn.model_selection import ParameterGrid
from utils import train 
from HySpecLab.unmixing import ContrastiveUnmixing
import pandas as pd
from tqdm import tqdm

n_bands = dataset.n_bands
results = pd.DataFrame(columns=['sigma_sparsity', 'sparse_weight', 'mean_rmse_x', 'mean_rmse_a', 'mean_sad_m'])
for params in ParameterGrid(param_grid):
    sigma = params['sigma_sparsity']
    sparse_weight = params['sparse_weight']
    print('Sigma:', sigma, 'Sparse Weight:', sparse_weight)
    batch_rmse_x, batch_rmse_a, batch_sad_m = [], [], []

    for i in tqdm(range(3)):
        model = ContrastiveUnmixing(n_bands, n_endmembers, endmember_init=logit_endmember_init, sigma_sparsity=sigma)
        train(model, n_endmembers, dataset, n_batchs=50, n_epochs=100, lr=1e-3, similarity_weight=1, sparse_weight=sparse_weight)
        rmse_x, rmse_a, sad_m = test(model, dataset)
        batch_rmse_x.append(rmse_x)
        batch_rmse_a.append(rmse_a)
        batch_sad_m.append(sad_m)

    mean_rmse_x = sum(batch_rmse_x) / len(batch_rmse_x)
    mean_rmse_a = sum(batch_rmse_a) / len(batch_rmse_a)
    mean_sad_m = sum(batch_sad_m) / len(batch_sad_m)
    
    results = results.append({'sigma_sparsity': sigma, 'sparse_weight': sparse_weight, 'mean_rmse_x': mean_rmse_x, 'mean_rmse_a': mean_rmse_a, 'mean_sad_m': mean_sad_m}, ignore_index=True)
    print('Mean RMSE X:', mean_rmse_x, 'Mean RMSE A:', mean_rmse_a, 'Mean SAD M:', mean_sad_m)

In [ ]:
results

In [ ]:
csv_save_path = os.path.join(result_path, 'hyperparameters.csv')
results.to_csv(csv_save_path, index=False)

In [ ]:
M_hat = sigmoid(model.ebk.detach().cpu()).numpy()

fig = plot_endmembers(M_hat, dataset.wv, ticks_range=(0, 1))
plt.show(fig)

In [ ]:
# test = model._sparse.detach().cpu().numpy().reshape(dataset.n_row, dataset.n_col)
test = model.sparse_gate.variational_parameter().detach().cpu().numpy().reshape(dataset.n_row, dataset.n_col)
test = np.log(test)
fig = plt.figure(figsize=(6,4))
plt.imshow(test.T, cmap='jet')
plt.grid(False)
# remove ticks
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar()
cbar.set_label(f'$\\log(\\tau)$', rotation=90, labelpad=8, fontsize='x-large')
# title to the colorbar
plt.show()

# fig.savefig(os.path.join(result_path, f'clhu/imgs/sparsity_estimation_{endmember_init_method}.pdf'), dpi=300, bbox_inches='tight')


In [ ]:
X_true = dataset.A @ dataset.endmembers()
X_hat = model(dataset.X).detach().cpu()
A_hat = torch.softmax(model.A.detach().cpu(), dim=1)
M_hat = sigmoid(model.ebk.detach().cpu())

real_M_hat = model(M_hat).detach()

import pandas as pd
df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
df['Method'] = ['CLHU']
df['RMSE_X'] = [rmse(X_true, X_hat, dim=None).numpy()]
df['RMSE_A'] = [rmse(dataset.A, A_hat, dim=None).numpy()]

# sad_result = sad(M_hat, dataset.endmembers()).numpy()
sad_result = sad(real_M_hat, dataset.endmembers()).numpy()
df['SAD_M'] = np.diagonal(sad_result).mean()

# df.to_csv(os.path.join(result_path, 'clhu/metrics.csv'), index=False)
df

In [ ]:
# model.ebk = sigmoid(model.ebk)
x_test = sigmoid(model.ebk)
m = model.ebk.detach()
model.eval()
print(x_test.min(), x_test.max())
encoder, projection = model.encoder, model.projection
from torch.nn.functional import normalize, softmax

def __similarity(X: torch.Tensor, ebk, temperature=1e-2) -> torch.Tensor:
        '''
            Cosine similarity between input and endmember bank.

            Parameters
            ----------
                x: torch.Tensor, shape=(batch_size, n_bands)
                    input tensor.
                
                temperature: float, default=1e-1
                    temperature parameter for contrastive learning.
                
        '''
        bs, n_bands = X.shape
        X = normalize(X, dim=1)

        normalize_ebk = normalize(ebk.detach(), dim=1).expand(bs, -1, -1)
        cos = torch.bmm(X.view(bs, 1, n_bands), torch.transpose(normalize_ebk, 1, 2)).squeeze()
        # return (1 - torch.pow(cos, 2))/temperature
        # return (torch.pow(cos, 2))/temperature
        return torch.log(torch.pow(cos, 2))

# y = projection(encoder(x_test))
# print(softmax(__similarity(y, m, temperature=.1).detach(), dim=1))

test = model(x_test)

plt.plot(test[2].detach().numpy())
plt.plot(x_test[2].detach().numpy())
plt.show()

softmax(model.A)

# ...

In [ ]:
from torch import sigmoid
def test(model, dataset):
    X = dataset.X
    model.eval()
    model = model.cpu()
    
    X_true = dataset.A @ dataset.endmembers()
    with torch.no_grad():
        X_hat = model(dataset.X)
        A_hat = torch.softmax(model.A, dim=1)
        M_hat = sigmoid(model.ebk) 
    
    rmse_x = rmse(X_true, X_hat, dim=None).numpy()      
    rmse_a = rmse(dataset.A, A_hat, dim=None).numpy()
    sad_m = np.diagonal(sad(M_hat, dataset.endmembers()).numpy()).mean()
    return rmse_x.item(), rmse_a.item(), sad_m

In [ ]:
from utils import train 
from HySpecLab.unmixing import ContrastiveUnmixing

n_bands = dataset.n_bands

batch_rmse_x = []
batch_rmse_a = []
batch_sad_m = []
for i in range(10):
    model = ContrastiveUnmixing(n_bands, n_endmembers, endmember_init=logit_endmember_init, sigma_sparsity=.25)
    train(model, n_endmembers, dataset, n_batchs=100, n_epochs=100, lr=1e-3, similarity_weight=1, sparse_weight=.5)

    rmse_x, rmse_a, sad_m = test(model, dataset)
    batch_rmse_x.append(rmse_x)
    batch_rmse_a.append(rmse_a)
    batch_sad_m.append(sad_m)

    print(rmse_x, rmse_a, sad_m)

In [ ]:
# generate dataframe
import pandas as pd
df = pd.DataFrame(columns=['RMSE_X', 'RMSE_A', 'SAD_M'])
df['RMSE_X'] = batch_rmse_x
df['RMSE_A'] = batch_rmse_a
df['SAD_M'] = batch_sad_m

# extract mean and std
df['RMSE_X'].mean(), df['RMSE_X'].std(), df['RMSE_A'].mean(), df['RMSE_A'].std(), df['SAD_M'].mean(), df['SAD_M'].std()

In [ ]:
df.to_csv(os.path.join(result_path, 'clhu/metrics_{}_batch.csv'.format(endmember_init_method)), index=False)

In [ ]:
os.path.join(result_path, 'clhu/metrics_{}_batch.csv'.format(endmember_init_method))

In [ ]:
plt.plot(sigmoid(model.ebk).T.detach().cpu().numpy())
plt.show()

In [ ]:
plt.plot(test.T.detach().cpu().numpy())
plt.show()

sad_m = np.diagonal(sad(test, dataset.endmembers()).detach().numpy()).mean()
sad_m

In [ ]:
from torch.nn.functional import normalize   
M = logit_endmember_init
M_norm = normalize(M, dim=1)

sim_matrix = torch.mm(M_norm, M_norm.T) / .1

logit = torch.log((sim_matrix).softmax(dim=1))
lo2 = (sim_matrix).log_softmax(dim=1)

sim_matrix, torch.isclose(logit, lo2)


In [ ]:
M_2 = M.clone()

M_2[0] = M_2[2]
M_norm = normalize(M_2, dim=1)

sim_matrix = torch.mm(M_norm, M_norm.T) / .1

logit = torch.log((sim_matrix).softmax(dim=1))
lo2 = (sim_matrix).log_softmax(dim=1)

sim_matrix, torch.isclose(logit, lo2)

# (sim_matrix).softmax(dim=1)

In [ ]:
import torch
from torch.nn.functional import dropout2d

X = torch.randn(1, 3, 3, 3)
hat_X = dropout2d(X, p=.5, training=True)